In [1]:
import pandas as pd
import numpy as np
import os
import statsmodels.api as sm

In [2]:

def process_file(file_path, text_columns_start_row=0):
    """
    Processes a text file to handle numeric columns and filter NaN values.
    
    Parameters:
    - file_path: Path to the text file.
    - text_columns_start_row: Row index from where to start processing text columns (default is 1).
    
    Returns:
    - recombined_df: A DataFrame with filtered numeric and text columns.
    """
    # Step 1: Load the data into a pandas DataFrame
    df = pd.read_csv(file_path, delimiter='\t', low_memory=False)
    
    # Step 2: Select the first two text columns
    df_first_two_columns = df.iloc[text_columns_start_row:, :2]  # First two columns
    
    # Step 3: Select the numeric columns (all except the first two and last two)
    df_numeric = df.iloc[text_columns_start_row:, 2:-2]  # Select numeric columns dynamically
    
    # Step 4: Apply to_numeric to convert non-numeric values to NaN. Exclude the description row and then add it back.
    first_row = df_numeric.iloc[0]
    rest_of_df = df_numeric.iloc[1:]
    df_numeric = rest_of_df.apply(pd.to_numeric, errors='coerce')
    df_numeric = pd.concat([pd.DataFrame([first_row]), df_numeric], ignore_index=True)

    # Step 5: Retain the last two text columns
    df_last_two_columns = df.iloc[text_columns_start_row:, -2:]  # Last two columns
    
    # Step 6: Filter rows with no NaN values in the numeric columns
    non_nan_filtered = df_numeric.dropna()
    
    # Step 7: Filter out columns that have all NaN values in the numeric subset

    # Function to check if all elements except the first one are NaN
    def should_drop(column):
        return column[1:].isna().all()

    # Identify columns to drop
    columns_to_drop = [col for col in df_numeric.columns if should_drop(df_numeric[col])]

    # Drop those columns
    non_nan_columns = df_numeric.drop(columns=columns_to_drop)
    
    # Step 8: Recombine the numeric subset, first two text columns, and last two text columns
    recombined_df = pd.concat([df_first_two_columns, non_nan_columns, df_last_two_columns], axis=1)

    # Step 9: Check that collection_title or promoted_subjectkey are both there, otherwise print a warning
    description_row = recombined_df.iloc[0]
    desc_contains_collection_title = description_row.apply(lambda x: 'collection_title' in str(x)).any()
    desc_contains_promoted_subjectkey = description_row.apply(lambda x: 'promoted_subjectkey' in str(x)).any()
    cols_contain_collection_title = 'collection_title' in recombined_df.columns
    cols_contain_promoted_subjectkey = 'promoted_subjectkey' in recombined_df.columns

    if not desc_contains_collection_title:
        print("Collection_title is missing from the column descriptions.")
    if not desc_contains_promoted_subjectkey:
        print("Promoted_subjectkey is missing from the column descriptions.")
    if not cols_contain_collection_title:
        print("Collection_title is missing from the column names.")
    if not cols_contain_promoted_subjectkey:
        print("Promoted_subjectkey is missing from the column names.")
    if desc_contains_collection_title and desc_contains_promoted_subjectkey and cols_contain_collection_title and cols_contain_promoted_subjectkey:
        return recombined_df

def process_all_files(root_folder, file_list, text_columns_start_row=0):
    """
    Processes all files in the list and returns the processed DataFrames.
    
    Parameters:
    - root_folder: Root folder containing all text files.
    - file_list: List of file names to process.
    - text_columns_start_row: Row index from where to start processing text columns (default is 1).
    
    Returns:
    - result_dict: Dictionary with file names as keys and processed DataFrames as values.
    """
    result_dict = {}
    
    for file_name in file_list:
        file_path = os.path.join(root_folder, file_name)
        print(f"Processing {file_name}...")
        
        # Process the file
        processed_df = process_file(file_path, text_columns_start_row)
        
        # Store the result in the dictionary
        if isinstance(processed_df, pd.DataFrame):
            result_dict[file_name] = processed_df
    
    return result_dict



In [3]:
#Get a list of all text files from the root folder

def get_text_files(directory):
    text_files = [f for f in os.listdir(directory) if f.endswith('.txt') and os.path.isfile(os.path.join(directory, f))]
    return text_files

root_folder = r"C:\Users\Senna\Desktop\Iigaya_lab\catie\catie_text_data\catie"
file_list = get_text_files(root_folder)
print(file_list)

['aesposys01.txt', 'aims01.txt', 'cata01.txt', 'cgis01.txt', 'clgry01.txt', 'dai01.txt', 'demo01.txt', 'dgsposys01.txt', 'dosecomp01.txt', 'ecg01.txt', 'endphase01.txt', 'endstudy01.txt', 'fint01.txt', 'hair01.txt', 'itaq01.txt', 'keyvars01.txt', 'lab01.txt', 'maccomp01.txt', 'macvlnce01.txt', 'med01.txt', 'meddispn01.txt', 'ndar_aggregate.txt', 'ndar_subject01.txt', 'neurobatt01.txt', 'package_info.txt', 'panss01.txt', 'qol01.txt', 'sae01.txt', 'scid_ph01.txt', 'screen01.txt', 'sf1201.txt', 'surf01.txt', 'surfq01.txt', 'timeto01.txt', 'viol01.txt', 'vitals01.txt']


In [4]:
# Process all files and store the results in a dictionary
processed_data = process_all_files(root_folder, file_list)
# processed_data['med01.txt']

Processing aesposys01.txt...
Processing aims01.txt...
Processing cata01.txt...
Processing cgis01.txt...
Processing clgry01.txt...
Processing dai01.txt...
Processing demo01.txt...
Processing dgsposys01.txt...
Processing dosecomp01.txt...
Processing ecg01.txt...
Processing endphase01.txt...
Processing endstudy01.txt...
Processing fint01.txt...
Processing hair01.txt...
Processing itaq01.txt...
Processing keyvars01.txt...
Processing lab01.txt...
Processing maccomp01.txt...
Processing macvlnce01.txt...
Processing med01.txt...
Processing meddispn01.txt...
Processing ndar_aggregate.txt...
Collection_title is missing from the column descriptions.
Promoted_subjectkey is missing from the column descriptions.
Collection_title is missing from the column names.
Promoted_subjectkey is missing from the column names.
Processing ndar_subject01.txt...
Processing neurobatt01.txt...
Processing package_info.txt...
Collection_title is missing from the column descriptions.
Promoted_subjectkey is missing from

In [5]:
#Merge all dfs which have promoted_subjectkey and a collection_title
#This is going to merge on all shared columns, doing outer to make sure all rows are kept. NaNs will go in where info does not exist for that row.

# Initial DataFrame (use the first DataFrame in the dictionary)
merged_df = list(processed_data.values())[0]

# Merge remaining DataFrames
for key in list(processed_data.keys())[1:]:
    merged_df = pd.merge(merged_df, processed_data[key], how='outer')

In [6]:
#other idea of what the column of medications for phase 1 is
#merged_df.loc['med2g']
merged_df['treat_1'].unique()

array(['Treatment for Phase 1', nan, 5.0, 1.0, 3.0, 2.0, 4.0],
      dtype=object)

In [7]:
# for each promoted subject key, get rows with non-NAN values of: treat_1, dcr_tae1, dcr_eff1
# with that promoted subject key as 'key', find the values for 'cocaine', 'opiates', 'pcp', 'meth', 'thc'.
# do logistic regression. 

In [8]:
# Step 1: Filter for non-NA 'treat_1' and get relevant 'promoted_subjectkey'
filtered_df = merged_df[merged_df['treat_1'].notna()]
promoted_keys = filtered_df['promoted_subjectkey'].unique()

In [9]:
pd.set_option('display.max_columns', None)
pd.set_option('display.max_rows', None)

# Display the first two rows
merged_df.head(2)

,collection_id,dataset_id,src_subject_id,visitid,copyid,truncvis,visday,anygenae,reportid,mdsev,ptsev,adh_imp,ptae,mdsev_c,mdae,bl1_vis,bl1b_vis,bl3_vis,bl2_vis,bl1_sev,bl1b_sev,bl2_sev,bl3_sev,bl1_psv,bl1b_psv,bl2_psv,bl3_psv,te1,te1b,te2,te3,pte1,pte1b,pte2,pte3,collection_title,promoted_subjectkey,aims_facial1_date1,aims_facial2_date1,aims_facial3_date1,aims_facial4_date1,aims_extrem5_date1,aims_extrem6_date1,aims_global8_date1,aims_global9_date1,aims_global10_date1,aims_dental11_date1,aims_dental12_date1,aims07,bar01,bar02,bar03,brsgca,sa01,sa02,sa03,sa04,sa05,sa06,criteria,facial,extremty,base1,base1b,base2,base3,last1,last1b,last2,last3,sevscore,brsobj,epsmean,b1_score,b1_index,b1_obj,b1_gca,b1_eps,b1b_scor,b1b_indx,b1b_obj,b1b_gca,b1b_eps,b2_score,b2_index,b2_obj,b2_gca,b2_eps,b3_score,b3_index,b3_obj,b3_gca,b3_eps,c_score,c_index,c_obj,c_gca,c_eps,si2_tep1,si1_tep1,gca_tep1,eps_tep1,pe2,pe2a,fucataa,fucataaa,fucataab,cs01,cs02a,cs02aa,cs02b,cs02ba,cs02c,cs02ca,cs02d,cs02da,cs02e,cs02ea,cs02f,cs02fa,cs03,cs04,cs05,cs06,cs07,cs08,cs09,cs10,cs11,cs12,cs13,cscigs,cs14,cs15,cs16,b1_cgis,b1_resp,c1_cgis,cgisresp,calg1,calg2,calg3,calg4,calg5,calg6,calg7,calg8,calg9,calg_ts,b1_calg,c1_calg,drug01,drug02,drug03,drug04,drug05,drug06,drug07,drug08,drug09,drug10,dai_ts,interview_age,white,black,native,race_c,hispanic,exacer,td,marital2,sigother,demo4a,educ_yrs,par_educ,demo5a,cgi_sev,cgi_cat,b1_panss,employ,employ2,qol01a,qol01b,qol01c,qol01d,qol01e,qol01f,qol01g,scid17a,yrs_trt,scid18a,yrs_pres,scid19,scid20,anti_beh,beh_cat,rescreen,scrn_dy,olz_0,quet_0,risp_0,zip_0,hal_0,deca_0,per_0,othr_0,allothr0,none_0,olzi_0,queti_0,rispi_0,dualmed0,allothi0,switch,me0a,me2,me3,me3a,any_mhx,status,b1_stat,b1b_stat,b2_stat,b3_stat,medad01,medad03,medad06,medad07,msa04,msa04b,medad08,medad10,medad11,medad13,medad14,capsules,dose1,dose2,dose3,presreg1,presreg2,avgcaps1,avgdose1,moddose1,catcaps1,avgcaps2,avgdose2,moddose2,catcaps2,avgdose3,moddose3,max1,max2,max3,ecgint,diag1,diag2,diag3,diag4,diag5,diag6,diag7,diag8,diag9,ecghr,ecgpr,ecgqrs,ecgqt,ecgqtc,ecgaxis,crfint,base11a_ecg,base1b_ecg,base2_ecg,base3_ecg,b1_qt,b1b_qt,b2_qt,b3_qt,ilast11a,ilast1b,ilast2,ilast3,qtlst11a,qtlst1b,qtlst2,qtlst3,csphase,c_qt,csprev,prolqtc1,rtd01,rtd02,rtd03,rtd04,rtd05a,rtd05b,rtd05c,rtd06a,rtd06b,rtd06c,rtd07a,rtd07c,rtd08,rtd09,rtd10,rtd11,rtd12,rtd13,rtd14,eos1,eos1a,eos1b,eos3,eos4,eos5,eos6,eos7,eos8,fi01,fi02,fi03,fi04,fi04a,fi05a,fi05b,fi05c,fi05d,fi05e,fi06,fi07,fi08,fi09,fi10,fi11,fi12,fi13,fi14,fi15,fi16,fi17,fi18,fi19,fi20,fi21,fi22,fi23,fi24,fi25,fi26,fi27a,fi27b,fi28a,fi28b,fi29a,fi29b,fi30a,fi30b,fi31a,fi31b,fi32a,fi32b,fi33a,fi33b,fi34a,fi34b,fi35a,fi35b,fi36a,fi36b,fi37a,fi37b,fi38a,fi38b,fi39a,fi39b,fi40a,fi40b,fi41,fi42,fi43,fi44,fi45,fi46,fi47,fi47a,fi48,fi48a,fi49,fi49a,fi50,fi50a,fi51,fi51a,fi52,fi53,fi54,fi55,fi56,fi57,fi58,fi59,fi60,fi61,fi62,fi63,fi64,fi65,fi66,fi67,fi68,fi68a,fi69,fi70,fi71,fi72a,fi72b,fi72c,fi72d,fi72e,fi72f,fi72g,fi73,fi74,fi75,fi76,fi77,fi78,fi79,fi80,fi81,fi82,fi83,fi84,fi85,fi86,fi87,fi88,fi89,fi90,fi91,fi92,fi93,fi94,fi95,fi96,fi97,fi98,fi99,fi100,fi101,fi102,fi103,fi104,fi105,fi106,fi107,fi108,fi109,fi110,fi111,fi112,fi113,fi114,fi115,fi116,fi117,fi117a,fi117b,fi117c,fi118,fi120,fi121,fi122,fi00b,colldy,length,cocaine,opiates,pcp,meth,thc,itaq01,itaq02,itaq03,itaq04,itaq05,itaq06,itaq07,itaq08,itaq09,itaq10,itaq11,itaq_ts,siteno,sitepool,scrnpop,multscrn,comp_s,dcr_s,es_day,in_fu,rand_1,rand_1a,rand1_1a,b1_day,e1_day,ittpop_1,comp_1,comp_1a,comp1_1a,dcr1_1,dcr2_1,rand_1b,b1b_day,e1b_day,ittpop1b,comp_1b,dcr1_1b,dcr2_1b,rand_2e,rand_2t,rand2,b2_day,e2_day,ittpop_2,comp_2e,comp_2t,comp2_et,dcr1_2,dcr2_2,in_3,b3_day,e3_day,comp_3,dcr1_3,dcr2_3,aedsc_3,phase1a,treat_1,treat_1a,treat11a,zip_1,zip_1a,zpr_cort,dose_1,dose_1a,randreg1,treat_1b,dose_1b,treat_2e,treat_2t,treat2,zip_2e,zip_2t,ph2_arm,dose_2e,dose_2t,randreg2,treat_31,treat_32,treat_3,set1,set2,set3,set4,repeat,result,hinorm,lonorm,cvresult,resn,cvhinor

In [10]:
# Step 2: Filter for the relevant 'promoted_subjectkey' and required columns
selected_df = merged_df[merged_df['promoted_subjectkey'].isin(promoted_keys)][[
    'promoted_subjectkey', 'dcr_tae1','dcr_eff1','facial', 'extremty', 'sevscore', 'brsobj', 'epsmean', 'dai_ts', 'calg_ts' ,'panss_positive'
]]
#
#dai: attitude toward med
#brs:mvmt disorder
#eps:extrapyramidal
#calgary: depression in schizophrenia

In [13]:
# Step 2b: Aggregate data by 'promoted_subjectkey'
aggregated_df = selected_df.groupby('promoted_subjectkey').agg({
    'dcr_tae1': 'first', 
    'dcr_eff1': 'first', 
    'facial': 'max',     
    'extremty': 'max',
    'sevscore': 'max',
    'brsobj': 'max',
    'epsmean': 'max',
    'dai_ts': 'max',
    'calg_ts': 'max',
    'panss_positive': 'max'
}).dropna(subset=['dcr_tae1'])  # Ensure 'dcr_tae1' has values



In [14]:
X = aggregated_df[['facial', 'extremty', 'sevscore', 'brsobj', 'epsmean', 'dai_ts', 'calg_ts' ,'panss_positive']].apply(pd.to_numeric, errors='coerce')
y_se = pd.to_numeric(aggregated_df['dcr_tae1'], errors='coerce')
y_eff = pd.to_numeric(aggregated_df['dcr_eff1'], errors='coerce')

In [15]:
data = pd.concat([X, y_se,y_eff], axis=1).dropna()
X = data[['facial', 'extremty', 'sevscore', 'brsobj', 'epsmean', 'dai_ts', 'calg_ts' ,'panss_positive']]
y_se = data['dcr_tae1']
y_eff = data['dcr_eff1']

In [16]:
# Add constant term for intercept
X = sm.add_constant(X)

# Fit logistic regression model sumptoms vs dcr_tae1
model = sm.Logit(y_se, X)
result = model.fit()

# Print the summary
print(result.summary())


# facial, calgary test, panss positive seems to be correlated, maybe dai too. 

Optimization terminated successfully.
         Current function value: 0.386542
         Iterations 6
                           Logit Regression Results                           
Dep. Variable:               dcr_tae1   No. Observations:                 1215
Model:                          Logit   Df Residuals:                     1206
Method:                           MLE   Df Model:                            8
Date:                Tue, 19 Nov 2024   Pseudo R-squ.:                 0.07853
Time:                        13:45:38   Log-Likelihood:                -469.65
converged:                       True   LL-Null:                       -509.67
Covariance Type:            nonrobust   LLR p-value:                 4.781e-14
                     coef    std err          z      P>|z|      [0.025      0.975]
----------------------------------------------------------------------------------
const             -2.4451      0.421     -5.801      0.000      -3.271      -1.619
facial           

In [17]:
# symptoms vs dcr_eff1

model = sm.Logit(y_eff, X)
result = model.fit()

print(result.summary())


Optimization terminated successfully.
         Current function value: 0.494426
         Iterations 6
                           Logit Regression Results                           
Dep. Variable:               dcr_eff1   No. Observations:                 1215
Model:                          Logit   Df Residuals:                     1206
Method:                           MLE   Df Model:                            8
Date:                Tue, 19 Nov 2024   Pseudo R-squ.:                 0.09397
Time:                        13:45:43   Log-Likelihood:                -600.73
converged:                       True   LL-Null:                       -663.04
Covariance Type:            nonrobust   LLR p-value:                 3.685e-23
                     coef    std err          z      P>|z|      [0.025      0.975]
----------------------------------------------------------------------------------
const             -4.8151      0.392    -12.276      0.000      -5.584      -4.046
facial           